#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2017


# Homework 2:  Link Analysis -- PageRank + SEO

### 100 points [5% of your final grade]

### Due: Tuesday, February 21, 2017 by 11:59pm

*Goals of this homework:* Explore real-world challenges of building a graph (in this case, from tweets), implement and test PageRank over this graph, and investigate factors that impact a page's rank on Google and Bing.

*Submission Instructions:* To submit your homework, rename this notebook as YOUR_UIN_hw2.ipynb. Submit this notebook via ecampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Friday, February 24, 2017 at 11:59pm.

# Part 1: PageRank (70 points)

## A Twitter-Mentioning Graph

In this assignment, we're going to adapt the classic PageRank approach to allow us to find not the most authoritative web pages, but rather to find significant Twitter users. So, instead of viewing the world as web pages with hyperlinks (where pages = nodes, hyperlinks = edges), we're going to construct a graph of Twitter users and their mentions of other Twitter users (so user = node, mention of another user = edge). Over this Twitter-user graph, we can apply the PageRank approach to order the users. The main idea is that a user who is mentioned by other users is more "impactful". 

Here is a toy example. Suppose you are given the following four tweets:

* **userID**: diane, **text**: "@bob Howdy!"
* **userID**: charlie, **text**: "Welcome @bob and @alice!"
* **userID**: bob, **text**: "Hi @charlie and @alice!"
* **userID**: alice, **text**: "Howdy!"

There are four short tweets generated by four users. The @mentions between users form a directed graph with four nodes and five edges. E.g., the "diane" node has a directed edge to the "bob" node. Note that a retweet also contain the "@", so it should be counted as a mention as well.

You should build a graph by parsing the tweets in the file we provide called *pagerank.json*.

**Notes:**

* The edges are binary and directed. If Bob mentions Alice once, in 10 tweets, or 10 times in one tweet, there is an edge from Bob to Alice, but there is not an edge from Alice to Bob.
* If a user mentions herself, ignore it.
* Correctly parsing @mentions in a tweet is error-prone. Use the entities field.
* Later you will need to implement the PageRank algorithm on the graph you build here.


In [1]:
# Here define your function for building the graph by parsing the input file of tweets
# Insert as many cells as you want
import json

data = []
with open('pagerank.json') as data_file:
    for line in data_file: 
        data.append(json.loads(line))
    
    

In [2]:
import collections

poster_l = []
mentionee_l = []

for user in data:
    user_id = user["user"]["id"]
    user_mentions = user["entities"]["user_mentions"]
    if user_mentions:
        for user in user_mentions:
            poster_l.append(user_id)
            mentionee_l.append(user["id"])

pair = zip(poster_l,mentionee_l)

graph = collections.defaultdict(set)
for k,v in pair:
    if not k==v:
        graph[k].add(v)



In [3]:
# Call your function to print out the size of the graph, i.e., the number of nodes and edges
# How you maintain the graph is totaly up to you
# However, if you encounter any memory issues, we recommend you write the graph into a file, and load it later.

def Node(graph):
    nodes = []
    a = graph.values()
    b = graph.keys()
    nodes += b
    
    for u in a:
        nodes += u
    nodes = list(set(nodes))
    
    return nodes

def Edge(graph):
    edge = 0
    a = graph.values()
    for l in a:
        edge += len(l)
    
    return edge
    

def Size(graph):
    n = Node(graph)
    e = Edge(graph)
    return (len(n),e)
    

In [4]:
print Size(graph)
# first is the number of nodes, second value is the number of edges

(16430, 24256)


We will not check the correctness of your graph. However, this will affect the PageRank results later.

## PageRank Implementation

Your program will return the top 10 users with highest PageRank scores. The **output** should be like:

* user1 - score1
* user2 - score2
* ...
* user10 - score10

You should follow these **rules**:

* Assume all nodes start out with equal probability.
* The probability of the random surfer teleporting is 0.1 (that is, the damping factor is 0.9).
* If a user is never mentioned and does not mention anyone, their PageRank scores should be zero. Do not include the user in the calculation.
* It is up to you to decide when to terminate the PageRank calculation.
* There are PageRank implementations out there on the web. Remember, your code should be **your own**.


**Hints**:
* If you're using the matrix style approach, you should use [numpy.matrix](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matrix.html).
* Scipy is built on top of Numpy and has support for sparse matrices. You most likely will not need to use Scipy unless you'd like to try out their sparse matrices.
* If you choose to use Numpy (and Scipy), please make sure your Anaconda environment include their latest versions (Numpy 1.12.0; Scipy 0.18.1).
* Test your parsing and PageRank calculations using a handful of tweets, before moving on to the entire file we provide.
* We will evaluate the user ranks you provide as well as the quality of your code. So make sure that your code is clear and readable.

What is the termination condition in your PageRank implementation? Describe it below:

After 100 iteration, the algorithm stopped. The reason is that iterating 50 time takes reasonable time to converge the top 10 ranks.

In [5]:
import numpy as np

def PageRanker(graph):
# generate the transition matrix
    m = Matrix(graph)
    transM = TransMatrix(m,0.9)

# generate the initial vector to represent the initial pagerank
    row = transM.shape[0]
    ini_matrix = (np.ones((row,1)))/row

# recursively process transistion matrix * current matrix
    cur_matrix = ini_matrix
    for i in range(0,50):
        cur_matrix = np.dot(transM,cur_matrix)
#pair the score with each file
    nodes = Node(graph)
    score_list = cur_matrix.flatten()
    pair = zip(nodes,score_list)
    sort_pair = sorted(pair, key=lambda a: a[1],reverse=True)
    
    return sort_pair[:10]
        
    

In [6]:
import numpy as np

def Matrix(graph):
    nodes= Node(graph)
    l = []
    length = len(nodes)
    keys = graph.keys()
# loop through the dictionary to get the key
    for i in range(0,length): 
        key = nodes[i]
    #  if that key exist,process the 0 and 1/len(items) for existence of items in dictionary
        if key in keys:
            items = graph[key]
            for node in nodes:
                if node in items:
                    l.append(1.0/len(items))
                else:
                    l.append(0)
    #  if the items do not exist, append length of 0 to l/length
        else:
            l += [1.0/length]*length
#     reshape the l to matrix
    m = np.array(l).reshape(length,length)
#     make the transpose of the matrix
    m = m.transpose()
    return m
    

In [7]:
def TransMatrix(matrix, d):
#     create hte uniform random matrix 
    shape = matrix.shape
    column = shape[0]
    a = np.ones(shape)
    a = a/column
# create the final transitional matrix
    m = (d*matrix)+((1-d)*a)
    return m
    

In [8]:
PageRanker(graph)

[(158314798, 0.0076887244092677508),
 (181561712, 0.0064985952268160211),
 (209708391, 0.0060516339215702869),
 (72064417, 0.0055161374888142338),
 (105119490, 0.0050649952569704459),
 (14268057, 0.0046614788690173027),
 (379961664, 0.0044081023112668028),
 (391037985, 0.0042409712356736625),
 (153074065, 0.0042376327811206038),
 (313525912, 0.0040796289430618492)]

# Part 2: Search Engine Optimization (30 + 5 points)

For this part, your goal is to put on your "[search engine optimization](https://en.wikipedia.org/wiki/Search_engine_optimization)" hat. Your job is to create a webpage that scores highest for the query: **awcv9kjlh scwrlkjf4e** --- two terms, lower case, no quote. As of today (Feb 7, 2017), there are no hits for this query on either Google or Bing. Based on our discussions of search engine ranking algorithms, you know that several factors may impact a page's rank. Your goal is to use this knowledge to promote your own page to the top of the list.

What we're doing here is a form of [SEO contest](https://en.wikipedia.org/wiki/SEO_contest). While you have great latitude in how you approach this problem, you are not allowed to engage in any unethical or illegal behavior. Please read the discussion of "white hat" versus "black hat" SEO over at [Wikipedia](https://en.wikipedia.org/wiki/Search_engine_optimization).


**Rules of the game:**

* Somewhere in the page (possibly in the non-viewable source html) you must include your name or some other way for us to identify you (e.g., your NetID, but not the UIN!).
* Your target page may only be a TAMU student page, a page on your own webserver, a page on a standard blog platform (e.g., wordpress), or some other primarily user-controlled page
* Your target page CAN NOT be a twitter account, a facebook page, a Yahoo Answers or similar page
* No wikipedia vandalism
* No yahoo/wiki answers questions
* No comment spamming of blogs
* If you have concerns/questions/clarifications, please post on Piazza and we will discuss

For your homework turnin for this part, you should provide us the URL of your target page and a brief discussion (2-4 paragraphs) of the strategies you are using. We will issue the query and check the rankings at some undetermined time in the next couple of weeks. You might guess that major search engines take some time to discover and integrate new pages: if I were you, I'd get a target page up immediately.

**Grading:**

* 5 points for providing a valid URL
* 20 points for a well-reasoned discussion of your strategy
* 5 points for your page appearing in the search results by Google or Bing (no matter how is the ranking)

** Bonus: **
* 1 point for your page appearing in the top-20 on Google or Bing
* 1 more point for your page appearing in the top-10 on Google or Bing
* 1 more point for your page appearing in the top-5 on Google or Bing
* 2 more points for your page being ranked first by Google or Bing. And, a vigorous announcement in class, and a high-five for having the top result!

What's the URL of your page?

http://people.tamu.edu/~xiemg/

What's your strategy? (2-4 paragraphs)

*ADD YOUR INPUT HERE*
1. Content based
    Increase the term frequency of the awcv9kjlh scwrlkjf4e.This will increase the score TF in TF-IDF
2. Add authority link reference link to make the website become the hub and increase the score of HIT
3. Upload a video on youtube and provide a link linked back to my website. Since youtube it is the fastest website to be recognized. The link back will increase the Authoritiy score in HIT and pagerank score
4. Make as many as possible website to direct to my website. Including my previous issue webpage which showcase my portfolilo. Also I let my friends' personal website to link to my webpage. This will boost up my score in pagerank and HIT algorithm.
